In [166]:
import pandas as pd 
import numpy as np
## display all the columns
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [167]:
##combine all the features into one dataset

In [168]:
movieDF = pd.read_csv('movie_df.csv')
twitterDF = pd.read_csv('twitter_bag_final.csv')
sentimentDF = pd.read_csv('sentimentresult.csv')

In [169]:
df1 = pd.merge(twitterDF,sentimentDF,how='inner',on='Movie')
merged_data = pd.merge(df1,movieDF,how='inner',left_on='Movie',right_on='movie_title')

In [170]:
#Step 1 check missing value
# Number of Nans
nan_list = merged_data.count() 
len(merged_data) - nan_list[nan_list < len(merged_data)]

aspect_ratio              50
budget                    30
runtime                    1
facenumber_in_poster       5
num_critic_for_reviews     2
gross                     76
overview                   1
color                      4
content_rating            22
language                   1
plot_keywords             25
dtype: int64

In [171]:
def get_average(data):
    avg = np.sum(data)/data.count()
    return avg

def get_common(data):
    mod = data.mode()
    return mod

In [172]:
# Step2 deal with missing value --- this is temporary method to do that
# duration num_critic_for_reviews, actor_2_facebook_likes,
# actor_3_facebook_likes,budget_x,runtime,duration,num_user_for_reviews
# fill with average
merged_data['duration'] = merged_data['duration'].fillna(get_average(merged_data['duration']))
merged_data['actor_2_facebook_likes'] = merged_data['actor_2_facebook_likes'].fillna(get_average(merged_data['actor_2_facebook_likes']))
merged_data['budget'] = merged_data['budget'].fillna(get_average(merged_data['budget']))
merged_data['runtime'] = merged_data['runtime'].fillna(get_average(merged_data['runtime']))
merged_data['duration'] = merged_data['duration'].fillna(get_average(merged_data['duration']))
merged_data['num_user_for_reviews'] = merged_data['num_user_for_reviews'].fillna(get_average(merged_data['num_user_for_reviews']))
merged_data['actor_3_facebook_likes'] = merged_data['actor_3_facebook_likes'].fillna(get_average(merged_data['actor_3_facebook_likes']))
merged_data['num_critic_for_reviews'] = merged_data['num_critic_for_reviews'].fillna(get_average(merged_data['num_critic_for_reviews']))

# facenumber_in_poster,language,color with the most common one
merged_data['facenumber_in_poster'] = merged_data['facenumber_in_poster'].fillna(get_common(merged_data['facenumber_in_poster'])[0])
merged_data['language'] = merged_data['language'].fillna(get_common(merged_data['language'])[0])
merged_data['color'] = merged_data['color'].fillna(get_common(merged_data['color'])[0])

# delete: gross, plot_keywords, content_rating, home_page, tagline, aspect_ratio fill
merged_data = merged_data.drop(columns = ['gross', 'plot_keywords', 'content_rating', 'aspect_ratio'\
                                         ,'actor_2_name','overview','actor_3_name'])

In [173]:
#Step 1 check missing value
# Number of Nans
nan_list = merged_data.count() 
len(merged_data) - nan_list[nan_list < len(merged_data)]

Series([], dtype: int64)

In [174]:
print(merged_data.columns)

Index(['Unnamed: 0_x', 'Unnamed: 0.1', 'content', 'Movie', 'Startdate',
       'Enddate', 'age', 'bad', 'best', 'big',
       ...
       'has_homepage', 'has_tagline', 'tagline_word_count', 'release_weekday',
       'release_month', 'release_year', 'keywords_count',
       'production_companies_count', 'production_countries_count',
       'spoken_languages_count'],
      dtype='object', length=159)


In [175]:
#drop features with sentence or words for baseline model
#also delete features which we cannot get before the movie shows
merged_data = merged_data.drop(columns = ['director_name','actor_1_name', 'movie_title','num_voted_users','movie_imdb_link','title_year',\
                           'imdb_score','id', 'keywords','original_language','original_title',\
                           'production_companies', 'production_countries', 'release_date','spoken_languages', 'title',
                            'vote_average', 'vote_count','Unnamed: 0_x','Unnamed: 0_y','content','Movie','Startdate','Enddate','text','Unnamed: 0'\
                                         ,'Unnamed: 0.1'])

In [176]:
#Step 4 mapping and one-hot encoding
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(merged_data.color)
merged_data.color = le.fit_transform(merged_data.color)

le1 = preprocessing.LabelEncoder()
le1.fit(merged_data.genres)
merged_data.genres = le1.fit_transform(merged_data.genres)

le2 = preprocessing.LabelEncoder()
le2.fit(merged_data.language)
merged_data.language = le2.fit_transform(merged_data.language)

le3 = preprocessing.LabelEncoder()
le3.fit(merged_data.country)
merged_data.country = le3.fit_transform(merged_data.country)

le4 = preprocessing.LabelEncoder()
le4.fit(merged_data.release_year)
merged_data.release_year= le4.fit_transform(merged_data.release_year)

In [177]:
merged_data.shape

(389, 132)

In [239]:
#feature selection
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(0.01)
selector.fit(merged_data)
reduced_features = [merged_data.columns[i] for i in range(len(selector.get_support(indices=False))) if selector.get_support(indices=False)[i] == False]
merged_data = merged_data.drop(columns = reduced_features)
print('The features whose variance is below the threshold are:\n', reduced_features)

The features whose variance is below the threshold are:
 []


In [260]:
from sklearn import datasets , linear_model
from sklearn.metrics import mean_squared_error , r2_score
from sklearn.model_selection import train_test_split

X = merged_data.drop(columns=['revenue'])
y = merged_data['revenue']

In [309]:
# Step 5 Trying a baseline model
X_train , X_test , y_train ,y_test = train_test_split(X,y,test_size=0.2 ,random_state=55)
LR = linear_model.LinearRegression()
LR.fit(X_train,y_train)
print('intercept_:%.3f' % LR.intercept_)
#print('coef_:%.3f' % LR.coef_)
print('Mean squared error: %.3f' % mean_squared_error(y_test,LR.predict(X_test)))
##((y_test-LR.predict(X_test))**2).mean()
print('Variance score: %.3f' % r2_score(y_test,LR.predict(X_test)))
#1-((y_test-LR.predict(X_test))**2).sum()/((y_test - y_test.mean())**2).sum()
print('score: %.3f' % LR.score(X_test,y_test))
#plt.scatter(X_test , y_test ,color ='green')
#plt.plot(X_test ,LR.predict(X_test) ,color='red',linewidth =3)
#plt.show()

intercept_:67329490.793
Mean squared error: 27503192631937856.000
Variance score: 0.665
score: 0.665


In [262]:
# Advanced Model
from sklearn.svm import SVR
import numpy as np
from sklearn.preprocessing import Normalizer

transformer = Normalizer().fit(X)
X1 = transformer.transform(X)
pca = PCA(n_components=50)
X2 = pca.fit_transform(X1)
X_train , X_test , y_train ,y_test = train_test_split(X1,y,test_size=0.2 ,random_state=55)

svr = SVR(kernel='poly', C=1000, degree=10)
svr.fit(X_train, y_train) 

#print('coef_:%.3f' % LR.coef_)
print('Mean squared error: %.3f' % mean_squared_error(y_test,svr.predict(X_test)))
##((y_test-LR.predict(X_test))**2).mean()
print('Variance score: %.3f' % r2_score(y_test,svr.predict(X_test)))
#1-((y_test-LR.predict(X_test))**2).sum()/((y_test - y_test.mean())**2).sum()
print('score: %.3f' % svr.score(X_test,y_test))


Mean squared error: 95363906588894608.000
Variance score: -0.162
score: -0.162


C:\Users\zhuwe\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [304]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
X2 = pca.fit_transform(X)
X_train , X_test , y_train ,y_test = train_test_split(X2,y,test_size=0.2 ,random_state=55)

In [305]:
from sklearn.ensemble import RandomForestRegressor
RF = RandomForestRegressor(max_depth=10, random_state=0,
                          n_estimators=100)
RF.fit(X_train, y_train) 
#print('coef_:%.3f' % LR.coef_)
print('Mean squared error: %.3f' % mean_squared_error(y_test,RF.predict(X_test)))
##((y_test-LR.predict(X_test))**2).mean()
print('Variance score: %.3f' % r2_score(y_test,RF.predict(X_test)))
#1-((y_test-LR.predict(X_test))**2).sum()/((y_test - y_test.mean())**2).sum()
print('score: %.3f' % RF.score(X_test,y_test))

Mean squared error: 27402006390378700.000
Variance score: 0.666
score: 0.666


In [306]:
from sklearn.ensemble import AdaBoostRegressor
adb = AdaBoostRegressor(random_state=0, n_estimators=1000)
adb.fit(X_train, y_train)  
adb.score(X_test, y_test) 
#print('coef_:%.3f' % LR.coef_)
print('Mean squared error: %.3f' % mean_squared_error(y_test,adb.predict(X_test)))
##((y_test-LR.predict(X_test))**2).mean()
print('Variance score: %.3f' % r2_score(y_test,adb.predict(X_test)))
#1-((y_test-LR.predict(X_test))**2).sum()/((y_test - y_test.mean())**2).sum()
print('score: %.3f' % adb.score(X_test,y_test))

Mean squared error: 29081076344593084.000
Variance score: 0.646
score: 0.646


In [307]:
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(random_state=0, n_estimators=1000)
gbm.fit(X_train, y_train)  
gbm.score(X_test, y_test) 
#print('coef_:%.3f' % LR.coef_)
print('Mean squared error: %.3f' % mean_squared_error(y_test,gbm.predict(X_test)))
##((y_test-LR.predict(X_test))**2).mean()
print('Variance score: %.3f' % r2_score(y_test,gbm.predict(X_test)))
#1-((y_test-LR.predict(X_test))**2).sum()/((y_test - y_test.mean())**2).sum()
print('score: %.3f' % gbm.score(X_test,y_test))

Mean squared error: 19848138293722380.000
Variance score: 0.758
score: 0.758
